<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Lionshead Fire Satellite Data Analysis

_by Joe Hamman & Jeremy Freeman (CarbonPlan), September 17, 2020_

This notebook performs analysis of the
[FIRMS Active Fire Dataset](https://firms.modaps.eosdis.nasa.gov/). Below we
calculate the fire perimeter for each day and overlay the total burned area to
with the impacted ACR260 credited forest offset project.

## Methodology

Here we take a simple and direct approach to calculated burned area from data
provided through the Fire Information for Resource Management System (FIRMS).

We begin with the active fire / hotspots shapefiles data (converted to GeoJSON
format) that can be
[downloaded from the FIRMS website](https://firms.modaps.eosdis.nasa.gov/download/).
We include data from the Modis (1km) and VIIRS 375m (Suomi-NPP and NOAA-20)
sensors.

These data come as point collections that must be merged to form polygons of
burned area. Here we apply a buffer on all points of 450 m before merging all
points together for each day. The result is a MultiPolygon actively burned area
for each day. When comparing the burned area to the ACR project, we further
merge all days into a single MultiPolygon.

A simplified set of MultiPolygons is saved at the end of the notebook for use in
mapping applications.

## References

- NRT VIIRS 375 m Active Fire product VJ114IMGTDL_NRT. Available on-line
  [https://earthdata.nasa.gov/firms]. doi: 10.5067/FIRMS/VIIRS/VJ114IMGT_NRT.002
- NRT VIIRS 375 m Active Fire product VNP14IMGT. Available on-line
  [https://earthdata.nasa.gov/firms].  
  doi:10.5067/FIRMS/VIIRS/VNP14IMGT_NRT.002
- MODIS Collection 6 NRT Hotspot / Active Fire Detections MCD14DL. Available
  on-line [https://earthdata.nasa.gov/firms]. doi:
  10.5067/FIRMS/MODIS/MCD14DL.NRT.006


In [ ]:
import copy
import os

from carbonplan_styles.colors import colors
import fsspec
import geopandas
import hvplot.pandas
import intake
import pandas as pd
import pyproj
import shapely
from shapely.ops import cascaded_union

# plot styles
style = "dark"
c = colors(style)
plot_opts = dict(geo=True, tiles="CartoDark", frame_width=800, project=True)

# Albers Equal Area Projection - we'll use this as a common projection to work in.
working_crs = pyproj.CRS(
    projparams="+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs"
)

# open our data catalog
cat = intake.open_catalog("./catalog.yaml")

write_to_azure = False

### Loading Fire data

As mentioned above, we've staged the FIRMS data in GeoJSON for ease of use.
Here, we load the three data files and concatenate them together.


In [ ]:
# open fire data
gdfs = []
for sensor in ["modis", "noaa", "suomi"]:
    temp = cat.firms(sensor=sensor).read()
    print(f"{sensor}: {len(temp)} records")
    gdfs.append(temp)

gdf = pd.concat(gdfs)

native_crs = copy.deepcopy(gdf.crs)  # we'll use this again at the end
gdf.head()

### Loading Project Polygons


In [ ]:
# The project geojson is missing its crs, so we're manually specifying it here.
project_crs = pyproj.CRS(
    'PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["false_easting",500000.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-123.0],PARAMETER["scale_factor",0.9996],PARAMETER["latitude_of_origin",0.0],UNIT["Meter",1.0]]'
)

# Open GeoJSON
project = cat.project.read()

# Set the CRS then re-project the data to our working CRS
project = project.set_crs(project_crs).to_crs(working_crs)

### Apply a buffer to all data points


In [ ]:
def points_to_pixels(points, buf=450):
    return [p.buffer(buf) for p in points]


gdf = gdf.set_geometry(
    points_to_pixels(gdf.geometry.to_crs(working_crs).values), crs=working_crs
)
gdf["date"] = pd.to_datetime(gdf["date"])

display(gdf.head())

### Merge all buffered points


In [ ]:
gdf = gdf.sort_index()

In [ ]:
merged_gdf = {}

for i, (date, df) in enumerate(gdf.groupby("date")):
    merged_gdf[date] = {
        "geometry": cascaded_union(df.geometry.values),
        "day": i,
    }
merged_gdf = geopandas.GeoDataFrame.from_dict(merged_gdf, orient="index")
merged_gdf = merged_gdf.set_geometry("geometry").set_crs(working_crs)

display(merged_gdf.head())

In [ ]:
tolerance = 50
full_res = merged_gdf.copy()  # keep a copy for the overlay with the project
merged_gdf = merged_gdf.set_geometry(
    merged_gdf.simplify(tolerance, preserve_topology=False)
)
merged_gdf["date"] = merged_gdf.index.strftime("%Y-%m-%d")
display(merged_gdf.head())

### Explore the daily burned area


In [ ]:
merged_gdf.reset_index().to_crs(native_crs).hvplot(
    color="date", alpha=0.5, cmap="hot", **plot_opts
)

### Merge all dates into a single burned area polygon


In [ ]:
burned_poly = cascaded_union(full_res.geometry.values)
burned_gdf = (
    geopandas.GeoDataFrame.from_dict(
        {"row": {"geometry": burned_poly}}, orient="index"
    )
    .set_geometry("geometry")
    .set_crs(working_crs)
)

### Overlay the burned area and project polygons


In [ ]:
burned_project = geopandas.overlay(burned_gdf, project, how="intersection")

### Explore the three polygon sets together


In [ ]:
opts = dict(**plot_opts)
del opts["tiles"]

plot = (
    burned_gdf.to_crs(native_crs).hvplot(
        color=c["red"], label="fire", tiles="CartoDark", line_color=None, **opts
    )
    * project.to_crs(native_crs).hvplot(
        color=c["text"], label="project", line_color=None, **opts
    )
    * burned_project.to_crs(native_crs).hvplot(
        color=c["green"], label="burned project", line_color=None, **opts
    )
)
plot

In [ ]:
# TODO: add comment confirming the project area matches here.
project_area = project.geometry.area.sum()
print("Project Area: %.2f m2" % project_area)

burned_project_area = burned_project.area.sum()
print("Burned Project Area: %.2f m2" % burned_project_area)
print(
    "Percent Project Burned: %.2f%%"
    % (burned_project_area / project_area * 100)
)

In [ ]:
# dump the polygons out to a geojson file for use in the article.
if write_to_azure:
    # blob file system
    from adlfs import AzureBlobFileSystem

    fs = AzureBlobFileSystem(
        account_name="carbonplan", account_key=os.environ["BLOB_ACCOUNT_KEY"]
    )
    with fs.open(
        "carbonplan-articles/offset-project-fire/lionshead_fire_polygons.json",
        "w",
    ) as f:
        f.write(merged_gdf.to_crs(native_crs).to_json())